In [1]:
from auto_reduce import *
import numpy as np

In [2]:
n = 4 # Number of states 
nouts = 2 # Number of outputs
# Either
# sys.load_SBML_model('my_sbml_model.xml')

# OR write ODEs
#      x = E, S, ES, P
#      P = a, d, k, Etot

# parameter values
P = np.zeros(4)
P[0] = 10
P[1] = 10
P[2] = 0.1
P[3] = 1
params_values = P.copy()
timepoints_ode = np.linspace(0, 100, 100) # timepoints for simulation

x_init = np.zeros(n) # Initial conditions
x_init[0] = 1
x_init[1] = 0.5

error_tol = 100
nstates_tol = 1
x, f, P = system.load_ODE_model(n, len(params_values))
params = P
# T1 and A1
f[0] = -P[0]*(P[3] - x[2])*x[1] + P[1]*x[2] + P[2]*x[2]
f[1] = -P[0]*(P[3] - x[2])*x[1] + P[1]*x[2]
f[2] = P[0]*(P[3] - x[2])*x[1] - (P[1] + P[2])*x[2]
f[3] = P[2]*x[2]

C = np.zeros((nouts,len(x)), dtype=int)
C[0][3] = 1
C = C.tolist()
sys = System(x, f, params = P, params_values = params_values, C = C, x_init = x_init)

In [6]:
from auto_reduce.utils import get_ODE
sys_ode = get_ODE(sys, timepoints_ode)
sol = sys_ode.solve_system().y
import matplotlib.pyplot as plt
plt.plot(timepoints_ode, np.transpose(np.array(C)@sol))
plt.xlabel('Time')
plt.ylabel('[Product]')
plt.show()

<Figure size 640x480 with 1 Axes>

In [3]:
from auto_reduce.utils import get_SSM
timepoints_ssm = np.linspace(0,100,10)
sys_ssm = get_SSM(sys, timepoints_ssm)
Ss = sys_ssm.compute_SSM() # len(timepoints) x len(params) x len(states)
out_Ss = np.zeros( (len(timepoints_ssm), len(params)) )
for i in range(len(params)):
    out_Ss[:,i] = (np.array(C)@(Ss[:,i,:].T))

In [4]:
import seaborn as sn
sn.heatmap(out_Ss.T)
plt.xlabel('Time')
plt.ylabel('Parameters')
plt.title('Sensitivity of output with respect to all parameters')
plt.show()

NameError: name 'plt' is not defined

In [3]:
from auto_reduce.utils import reduce
timepoints_ssm = np.linspace(0,100,10)
timepoints_ode = np.linspace(0, 100, 100)
sys_reduce = reduce(sys, timepoints_ode, timepoints_ssm)
results = sys_reduce.reduce_Cx()

Could not find solution for this collapsed variable : x0 from -P0*x1*(P3 - x2) + P1*x2 + P2*x2


NameError: name 'x2' is not defined

In [10]:
aa,bb = sys_reduce.get_reduced_system([0,1,2])

Could not find solution for this collapsed variable : x3 from P2*x2


In [14]:
aa.f

[-P0*x1*(P3 - x2) + P1*x2 + P2*x2,
 -P0*x1*(P3 - x2) + P1*x2,
 P0*x1*(P3 - x2) - x2*(P1 + P2)]

In [12]:
aa.x

[x0, x1, x2]